<a href="https://colab.research.google.com/github/RahulSrinivas8/deeplearning/blob/main/Text_Generation_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial demonstrates how to generate text using a character-based RNN.will work with a dataset of Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks. Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

#Setup

# Import Tensorflow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

# Download the Shakespeare dataset

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


# Read the data

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


#Process the text

#Vectorise the text

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)


In [ ]:
chars_from_ids = preprocessing.StringLookup(vocabulary = ids_from_chars.get_vocabulary(), invert=True, mask_token= None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

#Create training examples and targets


In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(15):
  print(chars_from_ids(ids).numpy().decode('UTF-8'))

F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [ ]:
seq_length =100
examples_per_epoch = len(text)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder= True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


it's easier to see what this is doing if you join the tokens back into string

In [ ]:
for seq in sequences.take(4):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"


In [ ]:
def split_input_target(sequences):
  input_text =sequences[:-1]
  target_text = sequences[1:]
  return input_text, target_text

In [ ]:
split_input_target(list("PYTHON"))

(['P', 'Y', 'T', 'H', 'O'], ['Y', 'T', 'H', 'O', 'N'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("input :",text_from_ids(input_example).numpy())
  print("target :",text_from_ids(target_example).numpy())

input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Creating Training Batches

In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build The Model

In [ ]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#Try the model

In [ ]:
# (first check the shape of output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  16896     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  67650     
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([49, 10, 13,  5, 14, 20, 34, 14, 61, 22, 42, 40, 46, 28, 27, 39, 41,
       57, 34, 57, 45,  8, 53, 57, 39, 11, 43, 33, 50, 25, 16,  6, 46, 61,
       42, 55, 45, 22, 40,  3, 43,  5, 10, 53, 41, 21, 51, 42, 16, 46, 41,
       30, 34,  6, 34, 52, 61, 63, 55, 63, 33, 16, 18, 31, 40, 32, 63, 63,
        3, 11,  7, 61, 33, 56, 35,  1,  1, 58, 14,  4, 62,  5, 23, 22, 47,
       51, 29, 15,  1, 53, 46, 29, 43, 40, 44, 30, 16, 30, 16,  2])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'mberland.\n\nNORTHUMBERLAND:\nThe commons will not then be satisfied.\n\nKING RICHARD II:\nThey shall be s'

Next Char Predictions:
 b"j3?&AGUAvIcagONZbrUrf-nrZ:dTkLC'gvcpfIa!d&3nbHlcCgbQU'UmvxpxTCERaSxx!:,vTqV\n\nsA$w&JIhlPB\nngPdaeQCQC "


#Train the model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.1903486


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

66.045815

In [ ]:
model.compile(optimizer='adam', loss=loss)

#Configure checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#Execute the training

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 35s 184ms/step - loss: 2.7687
Epoch 2/20
172/172 [==============================] - 33s 183ms/step - loss: 2.0207
Epoch 3/20
172/172 [==============================] - 33s 183ms/step - loss: 1.7359
Epoch 4/20
172/172 [==============================] - 33s 182ms/step - loss: 1.5688
Epoch 5/20
172/172 [==============================] - 33s 182ms/step - loss: 1.4647
Epoch 6/20
172/172 [==============================] - 33s 181ms/step - loss: 1.3936
Epoch 7/20
172/172 [==============================] - 33s 181ms/step - loss: 1.3395
Epoch 8/20
172/172 [==============================] - 33s 181ms/step - loss: 1.2934
Epoch 9/20
172/172 [==============================] - 33s 180ms/step - loss: 1.2524
Epoch 10/20
172/172 [==============================] - 32s 180ms/step - loss: 1.2131
Epoch 11/20
172/172 [==============================] - 33s 181ms/step - loss: 1.1738
Epoch 12/20
172/172 [==============================] - 32s 180ms/step - lo

#Generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Pourn if they do not much.

SAMPSON:
No, I come from Past self: for once would please your grace,
Thry govern proclaim'd it offended and
Destruction dagger, for your eye, that
mights said well accept: a watery from his bed,
And live an some fore, do me were so bootless.
I heard you seize and slaughter'd aims at said
Her and they the Ludlio's man, where we are gone, well, but
Their new commanings no sovereign.

KING HENRY VI:
Why, thing you went! The oath of man or wounded nothing of
thee thence from fortake to the crown o'clock.

KING HENRY VI:
Nay, take no man in him.

RICHARD:
And then, or else me now, speak from the hair
Bohes my report to the will;
For in a world so well aspections, to thy ears,
Will rather sweat of breathed upon,
And not unlikely like to fail in
Dire-dieving kinsman, no chrenkncil arms
We, how 'tis the murderer Provert: all's it is.

KING RICHARD II:
But when that mudder, long substitute,
Hath an thus can peer the vault, nor honour of glives:
One joins of t

If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nBlack for these good things to get you play.\nAs is the commission fairest flowers o' the east,\nYet thus thy boats we have but begun, prepare yourselves.\n\nKING RICHARD III:\nGood morrow, tender of the duke.\n\nDUKE VINCENTIO:\nUnbut a very little off,\nThough you have made he fight as you with a life\nThan dog wench; which of you would speed am not,\nLest thou himself will break the heavy child being\nMaliciously poison made thee provoked\nA borm of light to main'd you in her eyes.\nThey have most guarded; mistress beldow,\nWhilst thou hast made thy weary? never shame for it\nExceeding and stand to's a poor hell, friar.\nBoth to my among moieth to you that\nWhich hath as many dread: think it were, my kingry's books,\nShe shall count him in your comfort, wherein he speaks\nBeroaked in Padua shall have found: so do I;\nI for any laboure upon't, and not too coll\non her siste justful eye.\nA fox, ord their courses are but thy soul with chastisement;\nFor King of E